## Preamble

In this notebook, we run the Python script posted in the same folder to obtain the stock price data from Alphavantage API. We aim to extract the prices from October 2019 to November 2021 of each S&P500 index. Some indices have missing entries due to the availability of data from Alphavantage. 

We start by importing necessary packages.

In [9]:
import numpy as np
import os
import requests
from ediblepickle import pickle
from bs4 import BeautifulSoup
import pandas as pd
import csv
import sys
import subprocess
from urllib.parse import quote
from retrying import retry
from time import sleep
from ediblepickle import checkpoint

In [2]:
API_key = "MY2P7WF6CWPVBE7O"

In [3]:
cache_dir = 'cache'
if not os.path.exists(cache_dir):
    os.mkdir(cache_dir)

In [4]:
@checkpoint(key=lambda args, kwargs: quote(args[0]+'_'+args[1]) + '.pkl', work_dir=cache_dir)
@retry
def load_data(symbol,month):
    CSV_URL = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol='+\
    symbol+'&interval=1min&slice='+month+'&apikey='+API_key
    with requests.Session() as s:
        download = s.get(CSV_URL)
        decoded_content = download.content.decode('utf-8')
        cr = csv.reader(decoded_content.splitlines(), delimiter=',')
        data = list(cr)
        df = pd.DataFrame(data[1:],columns=data[0])
        if len(data) < 10:
            raise IOError("Failed attempt")
        else:
            return df

In [5]:
df = pd.read_csv('/Users/josht/Documents/GitHub/erdos_twitter_project/data/Stock_indices/snp500_list.csv')
tickers = df.Symbol

## Data Extraction

The cells below extract the data for tickers number 250 - 374 in S&P500. Data extraction for other tickers can be done in a similar fashion.

In [14]:
tickers[250: 376]

250    INTC
251     ICE
252     IBM
253      IP
254     IPG
       ... 
371     PFE
372      PM
373     PSX
374     PNW
375     PXD
Name: Symbol, Length: 126, dtype: object

Let's try aggregating the first one: `INTC`

In [36]:
df = load_data(tickers[250], "year1month1")

In [37]:
for j in [1,2]:
    for string in [f'year{j}month{k}' for k in range(3-j,13)]:
        df2 = load_data(tickers[250], string)
        df = pd.concat([df, df2], ignore_index=True)

In [38]:
df.to_csv("/Users/josht/Documents/GitHub/erdos_twitter_project/Stock_data_get/Stock_data_250_375/" + tickers[250] + ".csv")


Now, let's try on tickers 251 - 260

In [11]:
for i in range(251, 261):
    df = load_data(tickers[i], "year1month1")
    for j in [1,2]:
        kk = 3 - j
        for string in [f'year{j}month{k}' for k in range(3-j,13)]:
            df2 = load_data(tickers[i], string)
            df = pd.concat([df, df2], ignore_index=True)
            if kk % 4 == 0:
                print(tickers[i] + " " + string + " finished")
            kk += 1
    df.to_csv("/Users/josht/Documents/GitHub/erdos_twitter_project/Stock_data_get/Stock_data_250_375/" + tickers[i] + ".csv")
    
    

ICE year1month4 finished
ICE year1month8 finished
ICE year1month12 finished
ICE year2month4 finished
ICE year2month8 finished
ICE year2month12 finished
IBM year1month4 finished
IBM year1month8 finished
IBM year1month12 finished
IBM year2month4 finished
IBM year2month8 finished
IBM year2month12 finished
IP year1month4 finished
IP year1month8 finished
IP year1month12 finished
IP year2month4 finished
IP year2month8 finished
IP year2month12 finished
IPG year1month4 finished
IPG year1month8 finished
IPG year1month12 finished
IPG year2month4 finished
IPG year2month8 finished
IPG year2month12 finished
IFF year1month4 finished
IFF year1month8 finished
IFF year1month12 finished
IFF year2month4 finished
IFF year2month8 finished
IFF year2month12 finished
INTU year1month4 finished
INTU year1month8 finished
INTU year1month12 finished
INTU year2month4 finished
INTU year2month8 finished
INTU year2month12 finished
ISRG year1month4 finished
ISRG year1month8 finished
ISRG year1month12 finished
ISRG year

They're huge. Let's turn them into parquet files. 

In [12]:
for i in range(250, 261):
    df = pd.read_csv("/Users/josht/Documents/GitHub/erdos_twitter_project/Stock_data_get/Stock_data_250_375/" + tickers[i] + ".csv")
    df.to_parquet("/Users/josht/Documents/GitHub/erdos_twitter_project/Stock_data_get/Stock_data_250_375/" + tickers[i] + ".parquet")
    

Now, let's do the rest, also saving the tables as parquet files. 

In [7]:
ii = 1
for i in range(360, 375):
    df = load_data(tickers[i], "year1month1")
    for j in [1,2]:
        for string in [f'year{j}month{k}' for k in range(3-j,13)]:
            df2 = load_data(tickers[i], string)
            df = pd.concat([df, df2], ignore_index=True)
    df.to_parquet("/Users/josht/Documents/Stock_data_250_375/" + tickers[i] + ".parquet")
    print(str(ii) + ". " + tickers[i] + " finished")
    ii += 1


1. PCAR finished
2. PKG finished
3. PH finished
4. PAYX finished
5. PAYC finished
6. PYPL finished
7. PENN finished
8. PNR finished
9. PBCT finished
10. PEP finished
11. PKI finished
12. PFE finished
13. PM finished
14. PSX finished
15. PNW finished
